In [2]:
#coding = utf-8
import pandas as pd
import numpy as np
from operator import itemgetter
import math 
import json
import pickle
import re
import os
import csv
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfVectorizer
import logging
import jieba.posseg as pseg

In [3]:
root = r'C:\Users\Evilemon\Desktop\recommender\test-data\data'
result = r'C:\Users\Evilemon\Desktop\recommender\test-data\result'
wdir = os.path.join(root, '-ww')

In [5]:
def ItemUserMatrix(root,result):
    user_dict = dict()              # user_id 的数据字典
    jd_dict = dict()                #jd_poi 的数据字典
    jd_users_dict = dict()          #每个jd 对应的 去旅游过并进行评论过的用户的id
    user_jds_dict = dict()          #每个user去过并评论的jd的poi
    n = 0                           # user_id 的数据字典中的数字排序
    m = 0                           #jd_poi 的数据字典中的数字排序
    mat = np.full((16436,10),0.65)   #每一行代表用户对几个景点的评分数据
              
#    navg = 0
    #下面对文件夹中的数据进行遍历读取
    for dirpath, dirnames, filenames in os.walk(root):
        for filename in filenames:
            rpath = os.path.join(dirpath, filename)
            with open(rpath, 'rb') as cf:
                df = pd.DataFrame(pd.read_csv(cf, encoding="utf-8", lineterminator= '\n'))            
#            print(df._stat_axis.values.tolist())  #行名称
#            print(df.columns.values.tolist())     #列名称
#             K = 'ab'
#             print(IUD.setdefault('K'))
#             IUD[K] = set()
#             for user in df['user_id']:
#                 u = str(user)
#                 IUD[K].add(u)
            M = df.as_matrix()      #将Dataframe数据转换为矩阵形式
#            print(M.shape)
#            avg = np.mean(M[:,4])
#            print(avg)
#            navg += avg/5
#            print(navg)
            
            jd = str(M[3][3])     #将每个景点的jd_poi提取出来并转换成字符串
            x = 0                 #代表对每个不同的数据文件时都要从0行开始读取
            #jd的数据字典赋值
            if jd not in jd_dict.keys():   
                jd_dict[jd] = m
                
            #创建jd_users_dict字典
            if jd not in jd_users_dict.keys():
                jd_users_dict[m] = set()
            #读取user_id并传换成字符串，同时向jd_users_dict中添加用户
            for I in M[:,2]:
                
                if I not in user_jds_dict.keys():
                    user_jds_dict[n] = set()
               
                
                #user数据字典赋值，并且对mat赋值
                if I not in user_dict.keys():
                    user_dict[I] = n
                    jd_users_dict[m].add(n)
                    user_jds_dict[n].add(m)
                    a = 0
                    a = M[x][4]
                    b = a/5         #用最大值对评分做个正则化
                    mat[n][m] = b
                    n += 1
                    x += 1
                else:
                    c = user_dict[I]
                    jd_users_dict[m].add(c)
                    user_jds_dict[c].add(m)
                    a = 0
                    a = M[x][4]
                    b = a/5         #用最大值对评分做个正则化
                    mat[c][m] = b
                    x += 1
            m += 1
    #将得到的数据写入文件
    print(n)
    user_dict_path = os.path.join(result, 'user_dict')
    jd_dict_path = os.path.join(result, 'jd_dict')
    jd_users_dict_path = os.path.join(result, 'jd_users_dict')
    user_jds_dict_path = os.path.join(result, 'user_jds_dict')
    ItemUserMatrix_path = os.path.join(result, 'ItemUserMatrix')
    with open(user_dict_path, 'wb') as uf:
        pickle.dump(user_dict, uf)
    with open(jd_dict_path, 'wb') as jf:
        pickle.dump(jd_dict, jf)
    with open(jd_users_dict_path, 'wb') as juf:
        pickle.dump(jd_users_dict, juf)
    with open(user_jds_dict_path, 'wb') as ujf:
        pickle.dump(user_jds_dict, ujf)
    np.save(ItemUserMatrix_path,mat)
    return user_dict,jd_dict,jd_users_dict,user_jds_dict, mat
i,j,x,y,z = ItemUserMatrix(root,result)
#print(i['Uncle-Leiss'])
#print(j)
#print(x)
#print(y)
print(len(z[0]))

16436
10


In [4]:
def UserSimilarity(jd_users_dict,result):
    #caculate co-rated jds between users
    N = dict()       
    C = np.zeros((16436, 16436))
    for jd,users in jd_users_dict.items():
        for u in users:
            if u not in N:
                N[u] = 0
            N[u] += 1
            for v in users:
                if u == v:
                    continue
                C[u][v] += 1 / math.log(1+ len(users))
    #cacualte finial similarity matrix W
    W = np.zeros((16436, 16436))
    for u in range(len(C)):
        for v in range(len(C[0])):
            W[u][v] = C[u][v] / math.sqrt(N[u]* N[v])
    user_jds_num_dict = N                       #每个用户去过多少个 jd 的字典                                    
    user_jds_num_matrix = C                    #矩阵中每一个元素代表两用户都去过几个景点
    user_similarity_matrix = W                 #矩阵中的每一个元素代表两用户的相似度
    #将数据写入文件
    user_jds_num_dict_path = os.path.join(result, 'user_jds_num_dict')
    user_jds_num_matrix_path = os.path.join(result, 'user_jds_num_matrix')
    user_similarity_matrix_path = os.path.join(result, 'user_similarity_matrix')
    with open(user_jds_num_dict_path, 'wb') as junf:
        pickle.dump(user_jds_num_dict, junf)
    np.save(user_jds_num_matrix_path, user_jds_num_matrix)
    np.save(user_similarity_matrix_path, user_similarity_matrix)
    return W
W = UserSimilarity(x,result)
print(W[5000:5020,5000:5020])

[[0.         0.08446156 0.08446156 0.08446156 0.08446156 0.08446156
  0.08446156 0.08446156 0.08446156 0.08446156 0.08446156 0.08446156
  0.08446156 0.08446156 0.08446156 0.08446156 0.08446156 0.08446156
  0.08446156 0.08446156]
 [0.08446156 0.         0.11944668 0.11944668 0.11944668 0.11944668
  0.11944668 0.11944668 0.11944668 0.11944668 0.11944668 0.11944668
  0.11944668 0.11944668 0.11944668 0.11944668 0.11944668 0.11944668
  0.11944668 0.11944668]
 [0.08446156 0.11944668 0.         0.11944668 0.11944668 0.11944668
  0.11944668 0.11944668 0.11944668 0.11944668 0.11944668 0.11944668
  0.11944668 0.11944668 0.11944668 0.11944668 0.11944668 0.11944668
  0.11944668 0.11944668]
 [0.08446156 0.11944668 0.11944668 0.         0.11944668 0.11944668
  0.11944668 0.11944668 0.11944668 0.11944668 0.11944668 0.11944668
  0.11944668 0.11944668 0.11944668 0.11944668 0.11944668 0.11944668
  0.11944668 0.11944668]
 [0.08446156 0.11944668 0.11944668 0.11944668 0.         0.11944668
  0.11944668 0.1

In [5]:
def Recommendation(user_jds_dict, user_similarity_matrix, ItemUserMatrix, user_id, K):
    rank = dict()
    interacted_jds = user_jds_dict[user_id]
#    print((user_similarity_matrix[:,user_id]))
    y = user_similarity_matrix[:,user_id].argsort()
    for v in y[K:]:    #这里的K值为负，是从倒数 |K| 个向最后一个取值
        v = int(v)
#        print(v)
        for i in user_jds_dict[v]:
            i = int(i)
            if i not in interacted_jds:
                continue
            if i not in rank.keys():
                rank[i] = 0
            rank[i] += user_similarity_matrix[user_id][v] * ItemUserMatrix[v][i]
#            print(ItemUserMatrix[v][i])
    print(interacted_jds
    return rank
R = Recommendation(y, W,z, 0, -3)
print(sorted(R, reverse = True))

{0, 8, 3}
[8, 3, 0]


In [13]:
# def test():
#     user_dict = dict()
#     jd_dict = dict()
#     n = 0            # user_id 的数据字典
#     m = 0            #jd_poi 的数据字典
    
#     root = r'C:\Users\Evilemon\Desktop\recommender\test-data\data\八达岭长城-jd-comment.csv'
#     with open(root, 'rb') as cf:
#         df = pd.DataFrame(pd.read_csv(cf, encoding="utf-8", lineterminator= '\n')) 
#     M = df.as_matrix()
#     mat = np.zeros((1020,10))
#     jd = str(M[3,3])
#     x = 0
#     n = 0 
#     m = -1
#     if jd not in jd_dict.keys():
#         jd_dict[jd] = m
#         m += 1
#     for i in M[:,1]:
#         I = str(i)
#         if I not in user_dict.keys():
#             user_dict[I] = n
#             a = 0
#             a = M[x][4]
#             mat[n][m] = a
#             n += 1
#             x += 1     
#     return mat
# D = test()
# print(D)

[[4. 0. 0. ... 0. 0. 0.]
 [5. 0. 0. ... 0. 0. 0.]
 [5. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
